In [5]:
pip install pandas 

Note: you may need to restart the kernel to use updated packages.


In [6]:
import pickle
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error


In [4]:
from sklearn.pipeline import make_pipeline

In [5]:
pip install mlflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.2/28.2 MB 97.7 MB/s eta 0:00:002 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 94.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 92.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 MB 96.7 MB/s eta 0:00:000m eta 0:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 89.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 684.0/684.0 kB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 89.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 597.4/597.4 kB 37.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 70.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 88.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 79.8 MB/s eta 0:00:00


Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install boto3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 93.7 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.3.0
    Uninstalling urllib3-2.3.0:
      Successfully uninstalled urllib3-2.3.0
Note: you may need to restart the kernel to use updated packages.


In [8]:
import boto3
print(boto3.__version__)

1.37.30


In [9]:
import os

# Set the AWS profile to use
os.environ['AWS_PROFILE'] = 'ml_user'

In [10]:
import boto3

# Create a session and specify the region if necessary
session = boto3.Session(profile_name='ml_user', region_name='us-east-1')

# Create the S3 client and list buckets
s3_client = session.client('s3')
response = s3_client.list_buckets()
print(response)


{'ResponseMetadata': {'RequestId': 'TJZ36RKX9SZPX6PN', 'HostId': 'QMBxAMUcChS6yczi/UWalKVfT6JXdal+kxV8ZyYBQJHzzUS32luKE5H+OakCJI5+nnINiA7nGxM=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'QMBxAMUcChS6yczi/UWalKVfT6JXdal+kxV8ZyYBQJHzzUS32luKE5H+OakCJI5+nnINiA7nGxM=', 'x-amz-request-id': 'TJZ36RKX9SZPX6PN', 'date': 'Wed, 09 Apr 2025 16:28:32 GMT', 'content-type': 'application/xml', 'transfer-encoding': 'chunked', 'server': 'AmazonS3'}, 'RetryAttempts': 0}, 'Buckets': [{'Name': 'ml-sagemaker-practise', 'CreationDate': datetime.datetime(2025, 3, 5, 16, 9, 31, tzinfo=tzlocal())}, {'Name': 'mlflow-models-slv', 'CreationDate': datetime.datetime(2025, 3, 21, 15, 25, 50, tzinfo=tzlocal())}, {'Name': 'sagemaker-studio-585768144809-iwglcnciicp', 'CreationDate': datetime.datetime(2025, 1, 21, 20, 41, 34, tzinfo=tzlocal())}, {'Name': 'sagemaker-studio-585768144809-q9v9klv6dxe', 'CreationDate': datetime.datetime(2025, 1, 21, 20, 37, 14, tzinfo=tzlocal())}, {'Name': 'sagemaker-studio-58576

In [12]:
import sklearn
print(sklearn.__version__)


1.6.1


In [11]:
import mlflow
print(mlflow.__version__)


2.21.3


In [13]:
import mlflow
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("green taxi duration")

2025/04/09 16:33:01 INFO mlflow.tracking.fluent: Experiment with name 'green taxi duration' does not exist. Creating a new experiment.


<Experiment: artifact_location='s3://mlflow-models-slv/1', creation_time=1744216381043, experiment_id='1', last_update_time=1744216381043, lifecycle_stage='active', name='green taxi duration', tags={}>

In [14]:
def read_dataframe(filename: str):
    df=pd.read_parquet(filename)
    df['duration']=df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration=df.duration.apply(lambda m : m.total_seconds() / 60)
    
    df=df[(df.duration >= 1) & (df.duration <= 60)]
    categorical=['PULocationID','DOLocationID']
    df[categorical]=df[categorical].astype(str)
    return df    
    

In [15]:
def prepare_dictionaries(df:pd.DataFrame):
    df['PU_DO']=df['PULocationID'] + '_' + df['DOLocationID']
    categorical=['PU_DO']
    numerical=['trip_distance']
    dicts=df[categorical + numerical].to_dict(orient='records')
    return dicts

In [16]:
df_train=read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet')
df_val=read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet')
target='duration'
y_train=df_train[target].values
y_val=df_val[target].values

dict_train=prepare_dictionaries(df_train)
dict_val=prepare_dictionaries(df_val)

In [17]:
len(df_train),len(df_val)

(73908, 61921)

In [18]:
with mlflow.start_run():
    params=dict(max_depth=20,n_estimators=100,min_samples_leaf=10,random_state=0)
    mlflow.log_params(params)

    dv = DictVectorizer()
    model = RandomForestRegressor(**params,n_jobs=-1)

    X_train = dv.fit_transform(dict_train)
    model.fit(X_train,y_train)

    X_val = dv.transform(dict_val)
    y_pred = model.predict(X_val)

   
    mse = mean_squared_error(y_val, y_pred)
    print(params,mse)
    mlflow.log_metric('mse',mse)
    
    mlflow.sklearn.log_model(model,artifact_path="model")

    with open('dict_vectorizer.bin','wb') as f_out:
        pickle.dump(dv,f_out)
    mlflow.log_artifact('dict_vectorizer.bin')

{'max_depth': 20, 'n_estimators': 100, 'min_samples_leaf': 10, 'random_state': 0} 45.64114429815588


2025/04/09 16:33:57 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run capable-cub-981 at: http://127.0.0.1:5000/#/experiments/1/runs/391ccd9cc3b84136b80e2cc9fe82250b
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1


### using sklearn pipeline

In [19]:
with mlflow.start_run():
    params=dict(max_depth=20,n_estimators=100,min_samples_leaf=10,random_state=0)
    mlflow.log_params(params)
    pipeline=make_pipeline(
        DictVectorizer(),
        RandomForestRegressor(**params,n_jobs=-1)
    )
    pipeline.fit(dict_train,y_train)
    y_pred=pipeline.predict(dict_val)
    
    mse=mean_squared_error(y_pred,y_val)
    print(params,mse)
    mlflow.log_metric('mse',mse)
    mlflow.sklearn.log_model(pipeline,artifact_path="model")
    
    

{'max_depth': 20, 'n_estimators': 100, 'min_samples_leaf': 10, 'random_state': 0} 45.64114429815588


2025/04/09 16:34:42 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run gifted-gnu-737 at: http://127.0.0.1:5000/#/experiments/1/runs/6bf96782bee64c8cadefd3497b0712a1
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1


In [20]:
from mlflow.tracking import MlflowClient

In [23]:
MLFLOW_TRACKING_URI='http://127.0.0.1:5000'
RUN_ID='391ccd9cc3b84136b80e2cc9fe82250b'
client=MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [24]:
path=client.download_artifacts(run_id=RUN_ID,path='dict_vectorizer.bin')

In [25]:
with open(path,'rb')as f_out:
    dv=pickle.load(f_out)
          

In [26]:
dv

DictVectorizer()